In [39]:
import pandas as pd
import numpy as np
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from pandas import DataFrame
from matplotlib import pyplot
import string

In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/albert/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [119]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens
 
# load doc, clean and return line of tokens
def doc_to_line(content, vocab):
	# load the doc
	doc = content
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)

def process_docs(sentiment, vocab):
	lines = list()
	# walk through all files in the folder
	for index,row in sentiment.iterrows():
		line = doc_to_line(row['CONTENT'], vocab)
		# add to list
		lines.append(line)
	return lines

In [20]:
news_scraping_results = pd.read_csv("./news_scraping_results.csv")

In [21]:
news_scraping_results

,SYMBOL,SITE_NAME,URL,PUBLISH_TIME,SCRAPED_TIME,TITLE,CONTENT,sentiment
0,NKE,NaN,http://onlinenewsguru.com/2019/09/01/investors...,2019-09-01 07:00:00+00:00,2019-10-09-18-51,Investor's Roundup: Consolidated Water Co. Ltd...,george town cayman islands september 1 2019 sh...,neutral
1,C,NaN,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,2019-05-06 21:55:22+00:00,2019-10-09-18-51,CP Rail raises quarterly dividend 27.5% after ...,cp rail raise quarterly dividend 27 5 after st...,neutral
2,UFPT,NaN,http://www.bnnbloomberg.ca/unilever-subsidiary...,2019-04-09 07:00:00+00:00,2019-10-09-18-52,Unilever subsidiary Schmidt's Natural to launc...,unilever plc un n be dip -pron- toe into the c...,neutral
3,TER,NaN,http://www.businesskorea.co.kr/news/articleVie...,2019-01-24 08:00:00+00:00,2019-10-09-18-51,Multinational Pharma Companies Often Stop Drug...,the korean operation of japanese multinational...,neutral
4,EMMAW,GlobeNewswire,http://www.globenewswire.com/news-release/2019...,2019-07-18 07:00:00+00:00,2019-10-08-18-04,Telemynd Provides Update Following Spinoff Tra...,mission viejo calif and torrance calif july 18...,NaN
...,...,...,...,...,...,...,...,...
2569,BYND,NaN,https://www.usatoday.com/story/money/2019/10/0...,2019-10-09 13:33:00+00:00,2019-10-09-18-51,Johnson & Johnson ordered to pay $8 billion to...,close oklahoma lawyer argue johnson johnson ag...,neutral
2570,MSFT,NaN,https://www.wsj.com/articles/bright-horizons-t...,2019-04-05 07:00:00+00:00,2019-10-09-18-51,"Bright Horizons Tests Parents’ Patience, Not N...",for many thing app be great for other -pron- m...,neutral
2571,AHH,NaN,https://www.wtrf.com/local/main-street-bank-an...,2019-06-20 07:00:00+00:00,2019-10-09-18-52,Main Street Bank anniversary and new announcem...,a local business be celebrate -pron- 18th anni...,neutral
2572,TER,NaN,https://www.yahoo.com/entertainment/hgtvs-wind...,2019-07-10 07:00:00+00:00,2019-10-09-18-50,HGTV's Windy City Rehab Stars Have Work Permit...,the future of hgtv show windy city rehab be un...,neutral


In [163]:
positive_df = news_scraping_results[news_scraping_results['sentiment']=='positive']
positive_df = positive_df.reset_index(drop=True)
positive_df
n_positive = positive_df['sentiment'].values
n_positive[n_positive == 'positive'] = 1

In [164]:
negative_df = news_scraping_results[news_scraping_results['sentiment']=='negative']
negative_df = negative_df.reset_index(drop=True)
negative_df
n_negative = negative_df['sentiment'].values
n_negative[n_negative == 'negative'] = 0

In [75]:
'''neutral_df = news_scraping_results[news_scraping_results['sentiment']=='neutral']
neutral_df = neutral_df.reset_index(drop=True)
neutral_df
n_neutral = neutral_df['sentiment'].values
n_neutral[n_neutral == 'neutral'] = 2'''

,SYMBOL,SITE_NAME,URL,PUBLISH_TIME,SCRAPED_TIME,TITLE,CONTENT,sentiment
0,NKE,NaN,http://onlinenewsguru.com/2019/09/01/investors...,2019-09-01 07:00:00+00:00,2019-10-09-18-51,Investor's Roundup: Consolidated Water Co. Ltd...,george town cayman islands september 1 2019 sh...,neutral
1,C,NaN,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,2019-05-06 21:55:22+00:00,2019-10-09-18-51,CP Rail raises quarterly dividend 27.5% after ...,cp rail raise quarterly dividend 27 5 after st...,neutral
2,UFPT,NaN,http://www.bnnbloomberg.ca/unilever-subsidiary...,2019-04-09 07:00:00+00:00,2019-10-09-18-52,Unilever subsidiary Schmidt's Natural to launc...,unilever plc un n be dip -pron- toe into the c...,neutral
3,TER,NaN,http://www.businesskorea.co.kr/news/articleVie...,2019-01-24 08:00:00+00:00,2019-10-09-18-51,Multinational Pharma Companies Often Stop Drug...,the korean operation of japanese multinational...,neutral
4,CELG,GlobeNewswire,http://www.globenewswire.com/news-release/2019...,2019-07-23 07:00:00+00:00,2019-10-09-16-24,Jounce Therapeutics Announces Update on Strate...,celgene license jtx 8064 result in 50 0 millio...,neutral
...,...,...,...,...,...,...,...,...
1722,BYND,NaN,https://www.usatoday.com/story/money/2019/10/0...,2019-10-09 13:33:00+00:00,2019-10-09-18-51,Johnson & Johnson ordered to pay $8 billion to...,close oklahoma lawyer argue johnson johnson ag...,neutral
1723,MSFT,NaN,https://www.wsj.com/articles/bright-horizons-t...,2019-04-05 07:00:00+00:00,2019-10-09-18-51,"Bright Horizons Tests Parents’ Patience, Not N...",for many thing app be great for other -pron- m...,neutral
1724,AHH,NaN,https://www.wtrf.com/local/main-street-bank-an...,2019-06-20 07:00:00+00:00,2019-10-09-18-52,Main Street Bank anniversary and new announcem...,a local business be celebrate -pron- 18th anni...,neutral
1725,TER,NaN,https://www.yahoo.com/entertainment/hgtvs-wind...,2019-07-10 07:00:00+00:00,2019-10-09-18-50,HGTV's Windy City Rehab Stars Have Work Permit...,the future of hgtv show windy city rehab be un...,neutral


In [167]:
y = np.append(n_positive, n_negative)

In [102]:
def process_docs(sentiment, vocab):
    lines = list() 
    for index, row in sentiment.iterrows():
        add_doc_to_vocab(row['CONTENT'], vocab)
    return lines

# define vocab
vocab = Counter()
# add all docs to vocab
process_docs(positive_df, vocab)
process_docs(negative_df, vocab)
#process_docs(neutral_df, vocab)

[]

In [103]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]

In [104]:
# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()
 
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

In [99]:
# bow representation

# load doc, clean and return line of tokens
def doc_to_line(content, vocab):
	# load the doc
	doc = content
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)

# load all docs in a directory
def process_docs(sentiment, vocab):
	lines = list()
	# walk through all files in the folder
	for index,row in sentiment.iterrows():
		line = doc_to_line(row['CONTENT'], vocab)
		# add to list
		lines.append(line)
	return lines


"# load all docs in a directory\ndef process_docs(sentiment, vocab):\n\tlines = list()\n\t# walk through all files in the folder\n\tfor index,row in sentiment.iterrows():\n\t\tline = doc_to_line(row['CONTENT'], vocab)\n\t\t# add to list\n\t\tlines.append(line)\n\treturn lines"

In [112]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [121]:
# load all training reviews
positive_lines = process_docs(positive_df, vocab)
negative_lines = process_docs(negative_df, vocab)
#neutral_lines = process_docs(neutral_df, vocab)

In [122]:
# summarize what we have
print(len(positive_lines), 
      len(negative_lines) #,
      #len(neutral_lines)
     )

307 354


In [123]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
docs = positive_lines + negative_lines
tokenizer.fit_on_texts(docs)

In [130]:
tokenizer.texts_to_matrix(docs, mode='freq')

array([[0.        , 0.14814815, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03757225, 0.00578035, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03318584, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03826531, 0.00255102, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03174603, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05694761, 0.02961276, ..., 0.00455581, 0.00455581,
        0.00455581]])

In [131]:
# encode training data set
X = tokenizer.texts_to_matrix(docs, mode='freq')
print(X.shape)

(661, 12238)


In [169]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [170]:
np.shape(X_train)

(528, 12238)

In [171]:
np.shape(X_test)

(133, 12238)

In [172]:
np.shape(y_train)

(528,)

In [173]:
np.shape(y_test)

(133,)

In [174]:
y_train

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,

In [176]:
n_words = X_test.shape[1]

In [177]:
# define network
model = Sequential()
model.add(Dense(50, input_shape=(n_words,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [182]:
# fit network
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
 - 0s - loss: 0.1804 - accuracy: 0.9886
Epoch 2/50
 - 0s - loss: 0.1749 - accuracy: 0.9886
Epoch 3/50
 - 0s - loss: 0.1691 - accuracy: 0.9905
Epoch 4/50
 - 0s - loss: 0.1637 - accuracy: 0.9924
Epoch 5/50
 - 0s - loss: 0.1586 - accuracy: 0.9924
Epoch 6/50
 - 0s - loss: 0.1536 - accuracy: 0.9924
Epoch 7/50
 - 0s - loss: 0.1487 - accuracy: 0.9924
Epoch 8/50
 - 0s - loss: 0.1441 - accuracy: 0.9924
Epoch 9/50
 - 0s - loss: 0.1395 - accuracy: 0.9924
Epoch 10/50
 - 0s - loss: 0.1352 - accuracy: 0.9924
Epoch 11/50
 - 0s - loss: 0.1313 - accuracy: 0.9924
Epoch 12/50
 - 0s - loss: 0.1273 - accuracy: 0.9924
Epoch 13/50
 - 0s - loss: 0.1232 - accuracy: 0.9924
Epoch 14/50
 - 0s - loss: 0.1198 - accuracy: 0.9924
Epoch 15/50
 - 0s - loss: 0.1161 - accuracy: 0.9924
Epoch 16/50
 - 0s - loss: 0.1128 - accuracy: 0.9924
Epoch 17/50
 - 0s - loss: 0.1094 - accuracy: 0.9943
Epoch 18/50
 - 0s - loss: 0.1061 - accuracy: 0.9943
Epoch 19/50
 - 0s - loss: 0.1037 - accuracy: 0.9924
Epoch 20/50
 - 0s - l

In [183]:
# evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 66.165411


In [ ]:
modes = ['binary', 'count', 'tfidf', 'freq']
results = DataFrame()
for mode in modes:
	# evaluate model on data for mode
	results[mode] = evaluate_mode(X_train, y_train, X_test, y_test)
# summarize results
print(results.describe())


Epoch 1/50
 - 0s - loss: 0.6930 - accuracy: 0.5038
Epoch 2/50
 - 0s - loss: 0.6897 - accuracy: 0.5492
Epoch 3/50
 - 0s - loss: 0.6859 - accuracy: 0.5492
Epoch 4/50
 - 0s - loss: 0.6808 - accuracy: 0.5511
Epoch 5/50
 - 0s - loss: 0.6747 - accuracy: 0.5530
Epoch 6/50
 - 0s - loss: 0.6671 - accuracy: 0.5663
Epoch 7/50
 - 0s - loss: 0.6582 - accuracy: 0.5739
Epoch 8/50
 - 0s - loss: 0.6476 - accuracy: 0.6098
Epoch 9/50
 - 0s - loss: 0.6357 - accuracy: 0.6686
Epoch 10/50
 - 0s - loss: 0.6230 - accuracy: 0.7140
Epoch 11/50
 - 0s - loss: 0.6091 - accuracy: 0.7424
Epoch 12/50
 - 0s - loss: 0.5947 - accuracy: 0.8163
Epoch 13/50
 - 0s - loss: 0.5790 - accuracy: 0.8220
Epoch 14/50
 - 0s - loss: 0.5632 - accuracy: 0.8542
Epoch 15/50
 - 0s - loss: 0.5464 - accuracy: 0.8864
Epoch 16/50
 - 0s - loss: 0.5300 - accuracy: 0.8807
Epoch 17/50
 - 0s - loss: 0.5133 - accuracy: 0.9091
Epoch 18/50
 - 0s - loss: 0.4969 - accuracy: 0.9034
Epoch 19/50
 - 0s - loss: 0.4794 - accuracy: 0.9148
Epoch 20/50
 - 0s - l

Epoch 8/50
 - 0s - loss: 0.6402 - accuracy: 0.5701
Epoch 9/50
 - 0s - loss: 0.6296 - accuracy: 0.6004
Epoch 10/50
 - 0s - loss: 0.6182 - accuracy: 0.6610
Epoch 11/50
 - 0s - loss: 0.6062 - accuracy: 0.6572
Epoch 12/50
 - 0s - loss: 0.5936 - accuracy: 0.6856
Epoch 13/50
 - 0s - loss: 0.5803 - accuracy: 0.7311
Epoch 14/50
 - 0s - loss: 0.5666 - accuracy: 0.7614
Epoch 15/50
 - 0s - loss: 0.5522 - accuracy: 0.7879
Epoch 16/50
 - 0s - loss: 0.5378 - accuracy: 0.8277
Epoch 17/50
 - 0s - loss: 0.5233 - accuracy: 0.8295
Epoch 18/50
 - 0s - loss: 0.5092 - accuracy: 0.8788
Epoch 19/50
 - 0s - loss: 0.4940 - accuracy: 0.8561
Epoch 20/50
 - 0s - loss: 0.4792 - accuracy: 0.9034
Epoch 21/50
 - 0s - loss: 0.4639 - accuracy: 0.9394
Epoch 22/50
 - 0s - loss: 0.4489 - accuracy: 0.9129
Epoch 23/50
 - 0s - loss: 0.4332 - accuracy: 0.9527
Epoch 24/50
 - 0s - loss: 0.4191 - accuracy: 0.9394
Epoch 25/50
 - 0s - loss: 0.4037 - accuracy: 0.9602
Epoch 26/50
 - 0s - loss: 0.3892 - accuracy: 0.9583
Epoch 27/50
 -

Epoch 15/50
 - 0s - loss: 0.5219 - accuracy: 0.8902
Epoch 16/50
 - 0s - loss: 0.5052 - accuracy: 0.8920
Epoch 17/50
 - 0s - loss: 0.4864 - accuracy: 0.9280
Epoch 18/50
 - 0s - loss: 0.4689 - accuracy: 0.9261
Epoch 19/50
 - 0s - loss: 0.4510 - accuracy: 0.9375
Epoch 20/50
 - 0s - loss: 0.4338 - accuracy: 0.9413
Epoch 21/50
 - 0s - loss: 0.4174 - accuracy: 0.9545
Epoch 22/50
 - 0s - loss: 0.4007 - accuracy: 0.9527
Epoch 23/50
 - 0s - loss: 0.3843 - accuracy: 0.9621
Epoch 24/50
 - 0s - loss: 0.3694 - accuracy: 0.9659
Epoch 25/50
 - 0s - loss: 0.3545 - accuracy: 0.9659
Epoch 26/50
 - 0s - loss: 0.3401 - accuracy: 0.9659
Epoch 27/50
 - 0s - loss: 0.3263 - accuracy: 0.9678
Epoch 28/50
 - 0s - loss: 0.3136 - accuracy: 0.9754
Epoch 29/50
 - 0s - loss: 0.3003 - accuracy: 0.9716
Epoch 30/50
 - 0s - loss: 0.2877 - accuracy: 0.9811
Epoch 31/50
 - 0s - loss: 0.2757 - accuracy: 0.9792
Epoch 32/50
 - 0s - loss: 0.2644 - accuracy: 0.9811
Epoch 33/50
 - 0s - loss: 0.2535 - accuracy: 0.9811
Epoch 34/50


Epoch 22/50
 - 0s - loss: 0.4248 - accuracy: 0.9545
Epoch 23/50
 - 0s - loss: 0.4093 - accuracy: 0.9394
Epoch 24/50
 - 0s - loss: 0.3936 - accuracy: 0.9602
Epoch 25/50
 - 0s - loss: 0.3781 - accuracy: 0.9640
Epoch 26/50
 - 0s - loss: 0.3636 - accuracy: 0.9659
Epoch 27/50
 - 0s - loss: 0.3500 - accuracy: 0.9678
Epoch 28/50
 - 0s - loss: 0.3361 - accuracy: 0.9716
Epoch 29/50
 - 0s - loss: 0.3231 - accuracy: 0.9678
Epoch 30/50
 - 0s - loss: 0.3102 - accuracy: 0.9735
Epoch 31/50
 - 0s - loss: 0.2982 - accuracy: 0.9792
Epoch 32/50
 - 0s - loss: 0.2865 - accuracy: 0.9792
Epoch 33/50
 - 0s - loss: 0.2763 - accuracy: 0.9754
Epoch 34/50
 - 0s - loss: 0.2648 - accuracy: 0.9830
Epoch 35/50
 - 0s - loss: 0.2548 - accuracy: 0.9830
Epoch 36/50
 - 0s - loss: 0.2446 - accuracy: 0.9830
Epoch 37/50
 - 0s - loss: 0.2354 - accuracy: 0.9811
Epoch 38/50
 - 0s - loss: 0.2276 - accuracy: 0.9830
Epoch 39/50
 - 0s - loss: 0.2186 - accuracy: 0.9848
Epoch 40/50
 - 0s - loss: 0.2093 - accuracy: 0.9867
Epoch 41/50


Epoch 29/50
 - 0s - loss: 0.3273 - accuracy: 0.9735
Epoch 30/50
 - 0s - loss: 0.3150 - accuracy: 0.9697
Epoch 31/50
 - 0s - loss: 0.3020 - accuracy: 0.9754
Epoch 32/50
 - 0s - loss: 0.2909 - accuracy: 0.9754
Epoch 33/50
 - 0s - loss: 0.2797 - accuracy: 0.9811
Epoch 34/50
 - 0s - loss: 0.2699 - accuracy: 0.9811
Epoch 35/50
 - 0s - loss: 0.2588 - accuracy: 0.9811
Epoch 36/50
 - 0s - loss: 0.2494 - accuracy: 0.9830
Epoch 37/50
 - 0s - loss: 0.2406 - accuracy: 0.9811
Epoch 38/50
 - 0s - loss: 0.2309 - accuracy: 0.9830
Epoch 39/50
 - 0s - loss: 0.2219 - accuracy: 0.9830
Epoch 40/50
 - 0s - loss: 0.2145 - accuracy: 0.9848
Epoch 41/50
 - 0s - loss: 0.2060 - accuracy: 0.9867
Epoch 42/50
 - 0s - loss: 0.1991 - accuracy: 0.9867
Epoch 43/50
 - 0s - loss: 0.1917 - accuracy: 0.9867
Epoch 44/50
 - 0s - loss: 0.1849 - accuracy: 0.9886
Epoch 45/50
 - 0s - loss: 0.1780 - accuracy: 0.9905
Epoch 46/50
 - 0s - loss: 0.1711 - accuracy: 0.9886
Epoch 47/50
 - 0s - loss: 0.1651 - accuracy: 0.9905
Epoch 48/50


...

 - 0s - loss: 0.5041 - accuracy: 0.8996
Epoch 19/50
 - 0s - loss: 0.4874 - accuracy: 0.9186
Epoch 20/50
 - 0s - loss: 0.4715 - accuracy: 0.9129
Epoch 21/50
 - 0s - loss: 0.4557 - accuracy: 0.9205
Epoch 22/50
 - 0s - loss: 0.4398 - accuracy: 0.9413
Epoch 23/50
 - 0s - loss: 0.4251 - accuracy: 0.9318
Epoch 24/50
 - 0s - loss: 0.4107 - accuracy: 0.9527
Epoch 25/50
 - 0s - loss: 0.3944 - accuracy: 0.9527
Epoch 26/50
 - 0s - loss: 0.3799 - accuracy: 0.9583
Epoch 27/50
 - 0s - loss: 0.3663 - accuracy: 0.9640
Epoch 28/50
 - 0s - loss: 0.3526 - accuracy: 0.9640
Epoch 29/50
 - 0s - loss: 0.3393 - accuracy: 0.9678
Epoch 30/50
 - 0s - loss: 0.3264 - accuracy: 0.9735
Epoch 31/50
 - 0s - loss: 0.3142 - accuracy: 0.9716
Epoch 32/50
 - 0s - loss: 0.3023 - accuracy: 0.9754
Epoch 33/50
 - 0s - loss: 0.2910 - accuracy: 0.9792
Epoch 34/50
 - 0s - loss: 0.2799 - accuracy: 0.9792
Epoch 35/50
 - 0s - loss: 0.2697 - accuracy: 0.9830
Epoch 36/50
 - 0s - loss: 0.2594 - accuracy: 0.9848
Epoch 37/50
 - 0s - loss

Epoch 25/50
 - 0s - loss: 0.3992 - accuracy: 0.9564
Epoch 26/50
 - 0s - loss: 0.3847 - accuracy: 0.9621
Epoch 27/50
 - 0s - loss: 0.3705 - accuracy: 0.9640
Epoch 28/50
 - 0s - loss: 0.3574 - accuracy: 0.9640
Epoch 29/50
 - 0s - loss: 0.3438 - accuracy: 0.9659
Epoch 30/50
 - 0s - loss: 0.3320 - accuracy: 0.9716
Epoch 31/50
 - 0s - loss: 0.3199 - accuracy: 0.9659
Epoch 32/50
 - 0s - loss: 0.3073 - accuracy: 0.9754
Epoch 33/50
 - 0s - loss: 0.2953 - accuracy: 0.9792
Epoch 34/50
 - 0s - loss: 0.2849 - accuracy: 0.9773
Epoch 35/50
 - 0s - loss: 0.2740 - accuracy: 0.9811
Epoch 36/50
 - 0s - loss: 0.2636 - accuracy: 0.9811
Epoch 37/50
 - 0s - loss: 0.2539 - accuracy: 0.9811
Epoch 38/50
 - 0s - loss: 0.2453 - accuracy: 0.9811
Epoch 39/50
 - 0s - loss: 0.2362 - accuracy: 0.9830
Epoch 40/50
 - 0s - loss: 0.2270 - accuracy: 0.9867
Epoch 41/50
 - 0s - loss: 0.2196 - accuracy: 0.9867
Epoch 42/50
 - 0s - loss: 0.2108 - accuracy: 0.9867
Epoch 43/50
 - 0s - loss: 0.2034 - accuracy: 0.9886
Epoch 44/50


 - 0s - loss: 0.3358 - accuracy: 0.9678
Epoch 32/50
 - 0s - loss: 0.3226 - accuracy: 0.9754
Epoch 33/50
 - 0s - loss: 0.3109 - accuracy: 0.9716
Epoch 34/50
 - 0s - loss: 0.3001 - accuracy: 0.9754
Epoch 35/50
 - 0s - loss: 0.2890 - accuracy: 0.9773
Epoch 36/50
 - 0s - loss: 0.2787 - accuracy: 0.9792
Epoch 37/50
 - 0s - loss: 0.2690 - accuracy: 0.9811
Epoch 38/50
 - 0s - loss: 0.2589 - accuracy: 0.9830
Epoch 39/50
 - 0s - loss: 0.2500 - accuracy: 0.9830
Epoch 40/50
 - 0s - loss: 0.2410 - accuracy: 0.9848
Epoch 41/50
 - 0s - loss: 0.2323 - accuracy: 0.9867
Epoch 42/50
 - 0s - loss: 0.2242 - accuracy: 0.9886
Epoch 43/50
 - 0s - loss: 0.2163 - accuracy: 0.9886
Epoch 44/50
 - 0s - loss: 0.2091 - accuracy: 0.9886
Epoch 45/50
 - 0s - loss: 0.2016 - accuracy: 0.9886
Epoch 46/50
 - 0s - loss: 0.1943 - accuracy: 0.9886
Epoch 47/50
 - 0s - loss: 0.1878 - accuracy: 0.9886
Epoch 48/50
 - 0s - loss: 0.1815 - accuracy: 0.9905
Epoch 49/50
 - 0s - loss: 0.1752 - accuracy: 0.9905
Epoch 50/50
 - 0s - loss

Epoch 38/50
 - 0s - loss: 0.2052 - accuracy: 0.9848
Epoch 39/50
 - 0s - loss: 0.1963 - accuracy: 0.9867
Epoch 40/50
 - 0s - loss: 0.1887 - accuracy: 0.9886
Epoch 41/50
 - 0s - loss: 0.1814 - accuracy: 0.9886
Epoch 42/50
 - 0s - loss: 0.1740 - accuracy: 0.9924
Epoch 43/50
 - 0s - loss: 0.1675 - accuracy: 0.9924
Epoch 44/50
 - 0s - loss: 0.1609 - accuracy: 0.9924
Epoch 45/50
 - 0s - loss: 0.1547 - accuracy: 0.9924
Epoch 46/50
 - 0s - loss: 0.1494 - accuracy: 0.9924
Epoch 47/50
 - 0s - loss: 0.1431 - accuracy: 0.9924
Epoch 48/50
 - 0s - loss: 0.1379 - accuracy: 0.9924
Epoch 49/50
 - 0s - loss: 0.1329 - accuracy: 0.9924
Epoch 50/50
 - 0s - loss: 0.1279 - accuracy: 0.9924
28 accuracy: 0.6766917109489441
Epoch 1/50
 - 0s - loss: 0.6924 - accuracy: 0.5303
Epoch 2/50
 - 0s - loss: 0.6882 - accuracy: 0.5492
Epoch 3/50
 - 0s - loss: 0.6833 - accuracy: 0.5492
Epoch 4/50
 - 0s - loss: 0.6778 - accuracy: 0.5492
Epoch 5/50
 - 0s - loss: 0.6714 - accuracy: 0.5492
Epoch 6/50
 - 0s - loss: 0.6647 - acc

 - 0s - loss: 0.1826 - accuracy: 0.9886
Epoch 45/50
 - 0s - loss: 0.1763 - accuracy: 0.9905
Epoch 46/50
 - 0s - loss: 0.1698 - accuracy: 0.9867
Epoch 47/50
 - 0s - loss: 0.1635 - accuracy: 0.9886
Epoch 48/50
 - 0s - loss: 0.1579 - accuracy: 0.9924
Epoch 49/50
 - 0s - loss: 0.1517 - accuracy: 0.9905
Epoch 50/50
 - 0s - loss: 0.1467 - accuracy: 0.9924
1 accuracy: 0.6842105388641357
Epoch 1/50
 - 0s - loss: 0.6922 - accuracy: 0.5379
Epoch 2/50
 - 0s - loss: 0.6881 - accuracy: 0.5492
Epoch 3/50
 - 0s - loss: 0.6840 - accuracy: 0.5492
Epoch 4/50
 - 0s - loss: 0.6790 - accuracy: 0.5492
Epoch 5/50
 - 0s - loss: 0.6732 - accuracy: 0.5492
Epoch 6/50
 - 0s - loss: 0.6659 - accuracy: 0.5492
Epoch 7/50
 - 0s - loss: 0.6572 - accuracy: 0.5814
Epoch 8/50
 - 0s - loss: 0.6466 - accuracy: 0.5985
Epoch 9/50
 - 0s - loss: 0.6352 - accuracy: 0.6629
Epoch 10/50
 - 0s - loss: 0.6224 - accuracy: 0.7292
Epoch 11/50
 - 0s - loss: 0.6093 - accuracy: 0.7576
Epoch 12/50
 - 0s - loss: 0.5954 - accuracy: 0.8352
Ep

In [187]:
# evaluate a neural network model
def evaluate_mode(Xtrain, ytrain, Xtest, ytest):
	scores = list()
	n_repeats = 30
	n_words = Xtest.shape[1]
	for i in range(n_repeats):
		# define network
		model = Sequential()
		model.add(Dense(50, input_shape=(n_words,), activation='relu'))
		model.add(Dense(1, activation='sigmoid'))
		# compile network
		model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
		# fit network
		model.fit(Xtrain, ytrain, epochs=50, verbose=2)
		# evaluate
		loss, acc = model.evaluate(Xtest, ytest, verbose=0)
		scores.append(acc)
		print('%d accuracy: %s' % ((i+1), acc))
	return scores